In [8]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader



In [9]:
#create custom data class

class CustomDataset(Dataset):
    def __init__(self,low_idx,high_idx):
        self.data = pd.read_csv("./dataset/IRIS.csv.xls", sep=",", skiprows=range(low_idx,high_idx), header=1)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sepal_length = torch.tensor(self.data.iloc[idx, 0])
        sepal_width = self.data.iloc[idx, 1]
        petal_length = self.data.iloc[idx, 2]
        petal_width = self.data.iloc[idx, 3] 
        label = self.data.iloc[idx, 4]
        
        tensor_data = torch.tensor([sepal_length,sepal_width,sepal_width,sepal_width],dtype=torch.float32)
        
        return tensor_data , label

In [10]:
#just my afwull way to seperate training and testing dataset
# 80% training , 20% Test

train_dataset = CustomDataset(120,149)
test_dataset = CustomDataset(1,120)


FileNotFoundError: [Errno 2] No such file or directory: './dataset/IRIS.csv.xls'

In [ ]:
#create train_dataloader and test_dataloader
# 80% for training, 20% of data for testing -> 149*0.8 = 119.2 -> 120 , 29
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [5]:
#4 inputs and 3  outputs 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3),
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [6]:
#map labels into tensor of shape [1, 0, 0]
def transform_label(label_data):
    data = []
    for i in label_data:
        if i == "Iris-setosa":
            data.append(torch.tensor([1,0,0], dtype=torch.float32))
        elif i == "Iris-versicolor": 
            data.append(torch.tensor([0,1,0], dtype=torch.float32))
        else:
            data.append(torch.tensor([0,0,1], dtype=torch.float32))
    return torch.stack(data)

In [7]:
#train_labels, X_train = next(iter(train_loader))
X_train, train_labels = next(iter(train_loader))
train_labels = transform_label(train_labels)
X_train.shape, train_labels.shape, X_train.dtype, train_labels.dtype

(torch.Size([10, 4]), torch.Size([10, 3]), torch.float32, torch.float32)

In [8]:
X_train

tensor([[7.7000, 3.8000, 3.8000, 3.8000],
        [6.4000, 2.9000, 2.9000, 2.9000],
        [5.5000, 3.5000, 3.5000, 3.5000],
        [4.5000, 2.3000, 2.3000, 2.3000],
        [5.2000, 3.5000, 3.5000, 3.5000],
        [6.6000, 3.0000, 3.0000, 3.0000],
        [5.6000, 3.0000, 3.0000, 3.0000],
        [6.7000, 3.1000, 3.1000, 3.1000],
        [4.8000, 3.1000, 3.1000, 3.1000],
        [6.1000, 2.8000, 2.8000, 2.8000]])

In [9]:
train_labels

tensor([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.]])

In [10]:
#create model
lr = 0.01
model = NeuralNetwork()
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()
#Weights are by default torch.32 not 64 --> error message 

In [23]:
n_iters = 12
for epochs in range(n_iters):
    #forward pass and loss
    #test_labels, X_test = next(iter(test_loader))
    print(f"\n epochs: {epochs+1}/{n_iters} \n")
    for i,(inputs, labels) in enumerate(train_loader):
        out = model(inputs)
        train_labels = transform_label(labels)
        l = loss(out, train_labels)
        #backward pass
        l.backward()
        #update weights
        optim.step()
        optim.zero_grad()
    
        if i % 1 == 0:
            print(f" steps: {i+1}, loss : {l.item()}")


 epochs: 1/12 

 steps: 1, loss : 0.5263720750808716
 steps: 2, loss : 0.11929646879434586
 steps: 3, loss : 0.05268403887748718
 steps: 4, loss : 0.6870241761207581
 steps: 5, loss : 0.5492759346961975
 steps: 6, loss : 0.26037055253982544
 steps: 7, loss : 0.3575771450996399
 steps: 8, loss : 0.48591193556785583
 steps: 9, loss : 0.4868488907814026
 steps: 10, loss : 0.36501866579055786
 steps: 11, loss : 0.22247414290905
 steps: 12, loss : 0.5886343717575073

 epochs: 2/12 

 steps: 1, loss : 0.5818777084350586
 steps: 2, loss : 0.6087363958358765
 steps: 3, loss : 0.1646847426891327
 steps: 4, loss : 0.24342112243175507
 steps: 5, loss : 0.21152791380882263
 steps: 6, loss : 0.5987932085990906
 steps: 7, loss : 0.1915193349123001
 steps: 8, loss : 0.4551451802253723
 steps: 9, loss : 0.8736459612846375
 steps: 10, loss : 0.5212220549583435
 steps: 11, loss : 0.4451841711997986
 steps: 12, loss : 0.2829498052597046

 epochs: 3/12 

 steps: 1, loss : 0.2934994399547577
 steps: 2, lo

In [24]:
input_test = torch.tensor([4.9,3.1,1.5,0.1])

In [25]:
model(input_test)

tensor([-2.7823,  1.3695, -0.0747], grad_fn=<AddBackward0>)

In [127]:
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
X_test, test_labels = next(iter(train_loader))
test_labels = transform_label(test_labels)

In [128]:
x_label_predicted = model(X_test)

In [129]:
x_label_predicted

tensor([[ -4.5044,   1.5454,   0.7279],
        [  8.6874,  -1.8742,  -4.6533],
        [  9.9395,  -2.2701,  -5.2099],
        [-12.9874,   2.9779,   3.6421],
        [ 10.4682,  -2.4404,  -5.4439],
        [ -3.9118,   1.4826,   0.4582],
        [ -2.9685,   1.3415,   0.1030],
        [ -2.9268,   1.3456,   0.0688],
        [ -5.6717,   1.7588,   1.0973],
        [  7.4145,  -1.4370,  -4.0997]], grad_fn=<AddmmBackward0>)

In [130]:
_, x_label_predicted_hat = torch.max(x_label_predicted,1)

In [131]:
x_label_predicted_hat

tensor([1, 0, 0, 2, 0, 1, 1, 1, 1, 0])

In [132]:
def final_x(x_hat):
    list_tensors = []
    for i in x_hat:
        final_tensor = torch.zeros(1,3)
        final_tensor[0:,i] = 1
        list_tensors.append(final_tensor)
    return torch.cat(list_tensors)
    
    # tensor[0:,1] = 4 to accses specific value 

In [133]:
final_x = final_x(x_label_predicted_hat)

In [134]:
#final_x, test_labels
#print(f"{})

In [135]:
def get_accuracy(model_prediction, test_labels):
    idx = 0
    number_pred = 0
    counter = 0
    while idx < 10:
        print(f" \n Tensor[{idx+1}]")
        for i in range(3):
            print(f"model: {model_prediction[idx][i].item()}, label: {test_labels[idx][i].item()}")
            if model_prediction[idx][i].item() == test_labels[idx][i].item():

                counter += 1
                if counter == 3:
                    number_pred += 1
        counter = 0
          
        idx +=1
    accuracy = number_pred/10 *100
    return print(f"\naccuracy of model {number_pred/10 *100}%")

In [136]:
get_accuracy(final_x, test_labels)

 
 Tensor[1]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[2]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[3]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[4]
model: 0.0, label: 0.0
model: 0.0, label: 0.0
model: 1.0, label: 1.0
 
 Tensor[5]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[6]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[7]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[8]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[9]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[10]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0

accuracy of model 90.0%
